In [ ]:
!pip install -q transformers datasets accelerate bitsandbytes


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd

# Update the path if your file is somewhere else
csv_path = "/content/drive/MyDrive/..."

df = pd.read_csv(csv_path)

# Check the first few rows
df.head()


In [ ]:
import pandas as pd

# Load training set
train_df = pd.read_csv("/content/drive/MyDrive/...")

# Load test set and test labels
test_df = pd.read_csv("/content/drive/MyDrive/...")
test_labels_df = pd.read_csv("/content/drive/MyDrive/...")
# Merge test data and labels on 'id'
test_full = test_df.merge(test_labels_df, on="id")

# Keep only fully labeled examples (drop rows with any -1s)
label_cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
test_filtered = test_full[~test_full[label_cols].isin([-1]).any(axis=1)].copy()
from datasets import Dataset

# Prepare training dataset
train_df[label_cols] = train_df[label_cols].astype(int)
train_hf = Dataset.from_pandas(train_df[["comment_text"] + label_cols])

# Prepare eval dataset
test_filtered[label_cols] = test_filtered[label_cols].astype(int)
eval_hf = Dataset.from_pandas(test_filtered[["comment_text"] + label_cols])


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "answerdotai/ModernBERT-base"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6, trust_remote_code=True)


In [ ]:
def tokenize_and_format(example):
    # Tokenize the text
    tokens = tokenizer(example["comment_text"], padding="max_length", truncation=True, max_length=256)
    # Add multi-label targets
    tokens["labels"] = [
        float(example["toxic"]),
        float(example["severe_toxic"]),
        float(example["obscene"]),
        float(example["threat"]),
        float(example["insult"]),
        float(example["identity_hate"])
    ]
    return tokens

# Apply to both datasets
train_dataset = train_hf.map(tokenize_and_format)
eval_dataset = eval_hf.map(tokenize_and_format)

# Format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


In [ ]:
!pip install -q scikit-learn
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits)).numpy()

    # Threshold at 0.5
    preds = (probs >= 0.5).astype(int)

    # Flatten for metrics like accuracy
    labels_flat = labels.reshape(-1)
    preds_flat = preds.reshape(-1)
    probs_flat = probs.reshape(-1)

    return {
        "accuracy": accuracy_score(labels_flat, preds_flat),
        "f1_macro": f1_score(labels, preds, average="macro", zero_division=0),
        "f1_micro": f1_score(labels, preds, average="micro", zero_division=0),
        "roc_auc": roc_auc_score(labels, probs, average="macro"),
    }


In [ ]:
from transformers import Trainer
import torch.nn as nn
import torch

# Custom Trainer to compute multi-label classification loss
class MultiLabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fn = nn.BCEWithLogitsLoss()
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=3,  # ↑ from 1
    learning_rate=2e-5,
    warmup_steps=500,
    weight_decay=0.01,
    lr_scheduler_type="linear",
    fp16=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
)
trainer = MultiLabelTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()


In [ ]:


import pandas as pd
from sklearn.metrics import classification_report

# Assuming 'trainer' and 'eval_dataset' are defined from the previous code
predictions = trainer.predict(eval_dataset)

# Extract logits and labels
logits = predictions.predictions
labels = predictions.label_ids

# Convert logits to probabilities using sigmoid
probs = torch.sigmoid(torch.tensor(logits)).numpy()

# Threshold at 0.5 to get predictions
preds = (probs >= 0.5).astype(int)


# Generate the classification report
report = classification_report(labels, preds, target_names=label_cols, output_dict=True)

# Convert the dictionary to a DataFrame
report_df = pd.DataFrame(report).transpose()

# Save the DataFrame to a CSV file
report_df.to_csv('classification_report.csv')

from google.colab import files
files.download('classification_report.csv')

In [ ]:
save_path = "/content/drive/MyDrive/..."
# Save model
model.save_pretrained(save_path)

# Save tokenizer
tokenizer.save_pretrained(save_path)
